<h1><center> Welcome to AlignIt </center></h1>

Use this script to calculate symmetry deviation values, $\sigma _{ideal}$, between polyhedra with the same number of atoms. 

We use the formula:

<center> $S(G)=\frac{\sum_{k=1}^{N}{\left |Q_{k} - P_{k}\right |^{2}}}{\sum_{k=1}^{N}\left |Q_{k} - Q_{0}\right |^{2}}\cdot \frac{100}{N}$ </center>

Where Q and P is the two structures, k is each atom in the structure, N is the total number of atoms and $Q_{0}$ is zero point placed in origo of structure Q.

$(Q_{k} - Q_{0})^{2} $ normalises with the size of the structure, while $1 / N$ normalise with the number of atoms in the structure.



<h4><center> The formula is adapted from the following papers </center></h4>

Alvarez S., et al., Continuous symmetry maps and shape classification. The case of six-coordinated metal compounds, _New J. Chem._, <b>2002</b>, 26(8), 996-1009

Alvarez S., et al., Shape maps and polyhedral interconversion paths in transition metal chemistry, _Coordination Chemistry Reviews_, <b>2005</b>, 249(17-18), 1693-1708

Ruiz-Marínez A. & Alvarez S., Stereochemistry of Compounds with Coordination Number Ten, _Chemistry_, <b>2009</b>, 15(30), 7470-80


<h4><center> How to use AlignIt </center></h4>
A guide to calculating symmetry deviations values with AlignIt is found in Supporting Information of (reference to paper – in progress).

AlignIt uses manual alignment of atoms, bond length scaling and rotational optimization to calculate symmetry deviation values. For the latter, Mercury is used to overlay structures. The program is available for download at https://www.ccdc.cam.ac.uk/support-and-resources/Downloads/

Alternatively, Continuous Shape Measurement values can be calculated with SHAPE.  The program is available for download at http://www.ee.ub.edu/index.php?option=com_jdownloads&view=viewcategories&Itemid=529

<h2><center> Loading modules and $\sigma _{ideal}$ formula </center></h2>

In [1]:
import numpy as np
import pandas as pd
import os

def load_xyz(ideel_structure_name, disordered_structure_name, scaling, center_origo=True):
    """
    Load two XYZ files and center them around origo, scale them and output a np.array of the scaled coordinates.
    """
    ideel_structure = pd.read_csv(ideel_structure_name, skiprows=2, sep=" ", names=["atom", "x", "y", "z"])
    ideel_structure = np.concatenate((np.expand_dims((ideel_structure.x.values), axis=1), np.expand_dims((ideel_structure.y.values), axis=1), np.expand_dims((ideel_structure.z.values), axis=1)), axis=1)
    disordered_structure = pd.read_csv(disordered_structure_name, skiprows=2, sep=" ", names=["atom", "x", "y", "z"])
    print (disordered_structure)
    disordered_structure = np.concatenate((np.expand_dims((disordered_structure.x.values), axis=1), np.expand_dims((disordered_structure.y.values), axis=1), np.expand_dims((disordered_structure.z.values), axis=1)), axis=1)
    print (disordered_structure)
    
    # Center structure around origo
    if center_origo:
        ideel_structure[:,0] = ideel_structure[:,0] - ideel_structure[:,0].mean()
        ideel_structure[:,1] = ideel_structure[:,1] - ideel_structure[:,1].mean()
        ideel_structure[:,2] = ideel_structure[:,2] - ideel_structure[:,2].mean()
        disordered_structure[:,0] = disordered_structure[:,0] - disordered_structure[:,0].mean()
        disordered_structure[:,1] = disordered_structure[:,1] - disordered_structure[:,1].mean()
        disordered_structure[:,2] = disordered_structure[:,2] - disordered_structure[:,2].mean()

    # Scale structure
    disordered_structure *= scaling
    
    print ("ideel structure after centering:")
    print (ideel_structure)
    print ("Disordered structure after centering and scaling:")
    print (disordered_structure)

    return ideel_structure, disordered_structure

def symmetry_deviation_calculator(ideel_structure, disordered_structure):
    """
    Calculate symmetry deviation (𝜎) for two structures with equal amount of atoms.
    The 𝜎 is a number from 0 to 100, where 0 is completelly symmetric.
    This formula has been used from the papers:
    - Continuous symmetry maps and shape classification. The case of six-coordinated metal compounds
    - Shape maps and polyhedral interconversion paths in transition metal chemistry
    - Stereochemistry of Compounds with Coordination Number Ten

    The function needs two structure which is already aligned e.g. atom1 has to be the first atom in the list for both, atom2 has to be number 2 etc.
    The alignment can be done in mercury with the use of the overlay function.
    """
    
    dist = 0.0
    for atom in range(len(ideel_structure)):
        upper = np.linalg.norm(ideel_structure[atom] - disordered_structure[atom])**2
        lower = np.linalg.norm(ideel_structure[atom] - np.mean(ideel_structure.T, axis=1))**2
        together = upper/lower
        dist += together

    print ("𝜎 value: ")
    print (100 * dist / len(ideel_structure))
    return 100 * dist / len(ideel_structure)

def save_results(ideel_structure_name, disordered_structure_name, scaling, ideel_structure, disordered_structure, sigma, center_origo="True"):
    """
    Save results in folder --> Results_AlignIt
    """
    savePath = 'Results_AlignIt/'
    if not os.path.isdir(savePath): 
        os.mkdir(savePath)
        
    file = open(savePath+ideel_structure_name[:-4]+"_AND_"+disordered_structure_name[:-4]+".txt", 'w')
    file.write("Scaling = ")
    file.write(str(scaling) + "\n")
    file.write("Center origo = ")
    file.write(str(center_origo) + "\n")
    file.write(("Ideel structure:") + "\n")
    file.write(str(ideel_structure) + "\n")
    file.write(("Disordered structure:") + "\n")
    file.write(str(disordered_structure) + "\n")
    file.write("Symmetry deviation (sigma) value = ")
    file.write(str(sigma))
    file.close()
    
    return None


<h2><center> Define which structures $\sigma _{ideal}$ should be calculated on </center></h2>

In [2]:
ideel_structure_name = "triangle_symmetric.xyz"
disordered_structure_name = "triangle_disorder.xyz"
scaling = 1

In [5]:
ideel_structure, disordered_structure = load_xyz(ideel_structure_name, disordered_structure_name, scaling)
𝜎 = symmetry_deviation_calculator(ideel_structure, disordered_structure)

  atom       x    y       z
0   O1  0.0000  0.0  2.6350
1   O2  2.2820  0.0 -1.3175
2   O3 -2.0517  0.5 -1.3000
[[ 0.      0.      2.635 ]
 [ 2.282   0.     -1.3175]
 [-2.0517  0.5    -1.3   ]]
ideel structure after centering:
[[ 0.      0.      2.635 ]
 [ 2.282   0.     -1.3175]
 [-2.282   0.     -1.3175]]
Disordered structure after centering and scaling:
[[-0.07676667 -0.16666667  2.62916667]
 [ 2.20523333 -0.16666667 -1.32333333]
 [-2.12846667  0.33333333 -1.30583333]]
𝜎 value: 
0.9708601091029601


<h2><center> Save results </center></h2>

In [6]:
save_results(ideel_structure_name, disordered_structure_name, scaling, ideel_structure, disordered_structure, sigma=𝜎, center_origo="True")
